# Simple Email Security Demo

This notebook demonstrates basic email security scenarios with multi-agent interactions using PrimAITE-Mail.

## Scenarios Covered:
- Green Team: Normal email usage
- Red Team: Phishing and malicious emails
- Blue Team: Security responses and email blocking

All demonstrations use simple print statements and built-in display methods.

## Environment Setup

In [1]:
# Basic imports - no complex visualization dependencies
from primaite.simulator.sim_container import Simulation
from primaite.simulator.network.hardware.nodes.host.computer import Computer
from primaite.simulator.network.hardware.nodes.network.switch import Switch
from primaite_mail.simulator.software.smtp_server import SMTPServer
from primaite_mail.simulator.software.pop3_server import POP3Server
from primaite_mail.simulator.software.email_client import EmailClient
from primaite_mail.simulator.network.protocols.smtp import EmailMessage
from primaite import PRIMAITE_CONFIG

# Store original config to restore later
original_dev_mode = PRIMAITE_CONFIG["developer_mode"]["enabled"]
original_sys_logs = PRIMAITE_CONFIG["developer_mode"]["output_sys_logs"]
original_agent_logs = PRIMAITE_CONFIG["developer_mode"]["output_agent_logs"]

# Enable developer mode and system logs for debugging
PRIMAITE_CONFIG["developer_mode"]["enabled"] = True
PRIMAITE_CONFIG["developer_mode"]["output_sys_logs"] = True
PRIMAITE_CONFIG["developer_mode"]["output_agent_logs"] = True

print("✅ PrimAITE-Mail environment ready")
print("📧 Simple email security demo starting...")

✅ PrimAITE-Mail environment ready
📧 Simple email security demo starting...


## Basic Network Setup

In [2]:
# Create simple network topology
sim = Simulation()

# Create network switch
switch = Switch.from_config({
    "type": "switch",
    "hostname": "main_switch",
    "num_ports": 6,
    "operating_state": "ON"
})
sim.network.add_node(switch)

# Create mail server
mail_server = Computer.from_config({
    "type": "computer",
    "hostname": "mail_server",
    "ip_address": "192.168.1.10",
    "subnet_mask": "255.255.255.0",
    "operating_state": "ON"
})
sim.network.add_node(mail_server)

# Create user workstations
user_pc = Computer.from_config({
    "type": "computer",
    "hostname": "user_pc",
    "ip_address": "192.168.1.20",
    "subnet_mask": "255.255.255.0",
    "operating_state": "ON"
})
sim.network.add_node(user_pc)

admin_pc = Computer.from_config({
    "type": "computer",
    "hostname": "admin_pc",
    "ip_address": "192.168.1.21",
    "subnet_mask": "255.255.255.0",
    "operating_state": "ON"
})
sim.network.add_node(admin_pc)

# Create attacker machine
attacker_pc = Computer.from_config({
    "type": "computer",
    "hostname": "attacker_pc",
    "ip_address": "192.168.1.100",
    "subnet_mask": "255.255.255.0",
    "operating_state": "ON"
})
sim.network.add_node(attacker_pc)

# Connect all nodes to switch
sim.network.connect(mail_server.network_interface[1], switch.network_interface[1])
sim.network.connect(user_pc.network_interface[1], switch.network_interface[2])
sim.network.connect(admin_pc.network_interface[1], switch.network_interface[3])
sim.network.connect(attacker_pc.network_interface[1], switch.network_interface[4])

print("🌐 Network topology created:")
print(f"  📧 Mail Server: {mail_server.config.hostname} ({mail_server.config.ip_address})")
print(f"  👤 User PC: {user_pc.config.hostname} ({user_pc.config.ip_address})")
print(f"  👨‍💼 Admin PC: {admin_pc.config.hostname} ({admin_pc.config.ip_address})")
print(f"  🚨 Attacker PC: {attacker_pc.config.hostname} ({attacker_pc.config.ip_address})")

🌐 Network topology created:
  📧 Mail Server: mail_server (192.168.1.10)
  👤 User PC: user_pc (192.168.1.20)
  👨‍💼 Admin PC: admin_pc (192.168.1.21)
  🚨 Attacker PC: attacker_pc (192.168.1.100)


## Install Email Services

In [3]:
# Install email services on mail server
mail_server.software_manager.install(SMTPServer)
mail_server.software_manager.install(POP3Server)

smtp_server = mail_server.software_manager.software.get("smtp-server")
pop3_server = mail_server.software_manager.software.get("pop3-server")
pop3_server.mailbox_manager = smtp_server.mailbox_manager

# Install email clients on workstations
user_pc.software_manager.install(EmailClient)
admin_pc.software_manager.install(EmailClient)
attacker_pc.software_manager.install(EmailClient)

user_client = user_pc.software_manager.software.get("email-client")
admin_client = admin_pc.software_manager.software.get("email-client")
attacker_client = attacker_pc.software_manager.software.get("email-client")

# Create user mailboxes
users = ["user", "admin", "finance", "hr"]
for username in users:
    smtp_server.mailbox_manager.create_mailbox(username)

print("📧 Email services installed:")
print(f"  ✅ SMTP Server on port {smtp_server.port}")
print(f"  ✅ POP3 Server on port {pop3_server.port}")
print(f"  ✅ Email clients installed on all workstations")
print(f"  ✅ Created {len(users)} user mailboxes")

📧 Email services installed:
  ✅ SMTP Server on port 25
  ✅ POP3 Server on port 110
  ✅ Email clients installed on all workstations
  ✅ Created 4 user mailboxes


## Green Team Scenario: Normal Email Usage

Demonstrate legitimate email communication between users.

In [4]:
print("=== GREEN TEAM: Normal Email Usage ===")

# Configure legitimate users
admin_client.config.username = "admin@company.com"
admin_client.config.default_smtp_server = str(mail_server.config.ip_address)

user_client.config.username = "user@company.com"
user_client.config.default_smtp_server = str(mail_server.config.ip_address)

# Send normal business email
business_email = EmailMessage(
    sender="admin@company.com",
    recipients=["user@company.com"],
    subject="Weekly Team Meeting",
    body="Hi Team,\n\nOur weekly meeting is scheduled for Friday at 2 PM in Conference Room A.\n\nBest regards,\nAdmin"
)

success = admin_client.send_email(business_email, mail_server.config.ip_address)
print(f"📧 Business email sent: {'✅ Success' if success else '❌ Failed'}")
print(f"   From: {business_email.sender}")
print(f"   To: {business_email.recipients[0]}")
print(f"   Subject: {business_email.subject}")

# Send reply
reply_email = EmailMessage(
    sender="user@company.com",
    recipients=["admin@company.com"],
    subject="Re: Weekly Team Meeting",
    body="Thanks for the reminder! I'll be there.\n\nBest,\nUser"
)

success = user_client.send_email(reply_email, mail_server.config.ip_address)
print(f"📧 Reply email sent: {'✅ Success' if success else '❌ Failed'}")

# Show current mailbox contents using built-in method
print("\n📬 Current mailbox contents:")
smtp_server.show_mailbox("user")
print("")
smtp_server.show_mailbox("admin")

=== GREEN TEAM: Normal Email Usage ===
📧 Business email sent: ❌ Failed
   From: admin@company.com
   To: user@company.com
   Subject: Weekly Team Meeting
📧 Reply email sent: ❌ Failed

📬 Current mailbox contents:
user's mailbox is empty

admin's mailbox is empty


## Red Team Scenario: Phishing Attack

Simulate a phishing attack from the attacker machine.

In [5]:
print("=== RED TEAM: Phishing Attack ===")

# Configure attacker client
attacker_client.config.username = "attacker@external.com"
attacker_client.config.default_smtp_server = str(mail_server.config.ip_address)

# Create phishing email (spoofed sender)
phishing_email = EmailMessage(
    sender="admin@company.com",  # Spoofed sender
    recipients=["user@company.com", "finance@company.com"],
    subject="URGENT: Account Security Update Required",
    body="URGENT SECURITY NOTICE\n\nYour account has been compromised. Click here immediately to secure your account:\n\nhttp://malicious-phishing-site.com/login\n\nFailure to act within 1 hour will result in account suspension.\n\n'IT Security Team'"
)

print("🚨 Phishing attack launched:")
print(f"   From: {phishing_email.sender} (SPOOFED!)")
print(f"   To: {', '.join(phishing_email.recipients)}")
print(f"   Subject: {phishing_email.subject}")
print(f"   Actual source IP: {attacker_pc.config.ip_address}")

# Attempt to send phishing email
success = attacker_client.send_email(phishing_email, mail_server.config.ip_address)
print(f"📧 Phishing email sent: {'✅ Success' if success else '❌ Blocked'}")

if success:
    print("⚠️ SECURITY ALERT: Phishing email delivered to targets!")
    print("🎯 Attack successful - users may be compromised")
else:
    print("✅ SECURITY: Phishing email blocked by security measures")

# Show updated mailbox contents
print("\n📬 Mailbox contents after attack:")
smtp_server.show_mailbox("user")
print("")
smtp_server.show_mailbox("finance")

=== RED TEAM: Phishing Attack ===
🚨 Phishing attack launched:
   From: admin@company.com (SPOOFED!)
   To: user@company.com, finance@company.com
   Subject: URGENT: Account Security Update Required
   Actual source IP: 192.168.1.100
📧 Phishing email sent: ❌ Blocked
✅ SECURITY: Phishing email blocked by security measures

📬 Mailbox contents after attack:
user's mailbox is empty

finance's mailbox is empty


## Blue Team Scenario: Security Response

Demonstrate blue team defensive actions using email blocking.

In [6]:
print("=== BLUE TEAM: Security Response ===")

# Blue team detects the attack and responds
print("🔍 SOC analyst detects suspicious email activity...")
print(f"   Suspicious sender: admin@company.com (from {attacker_pc.config.ip_address})")
print(f"   Threat type: Email spoofing/phishing")

# Block the malicious sender
print("\n🛡️ Implementing security countermeasures...")
block_sender_request = [
    "network", "node", "mail_server", "service", "smtp-server", 
    "block_sender", {"sender_address": "admin@company.com"}
]

response = sim.apply_request(request=block_sender_request, context={})
print(f"📤 Block sender request: {response.status}")
if response.status == "success":
    print(f"✅ Sender blocked successfully")
    print(f"📊 Total blocked senders: {response.data.get('blocked_senders_count', 'N/A')}")

# Block the attacker's IP
block_ip_request = [
    "network", "node", "mail_server", "service", "smtp-server", 
    "block_ip", {"ip_address": str(attacker_pc.config.ip_address)}
]

response = sim.apply_request(request=block_ip_request, context={})
print(f"📤 Block IP request: {response.status}")
if response.status == "success":
    print(f"✅ IP {attacker_pc.config.ip_address} blocked successfully")
    print(f"📊 Total blocked IPs: {response.data.get('blocked_ips_count', 'N/A')}")

# Test if blocking is effective
print("\n🧪 Testing security measures...")
test_email = EmailMessage(
    sender="admin@company.com",
    recipients=["hr@company.com"],
    subject="Another phishing attempt",
    body="This should be blocked now."
)

success = attacker_client.send_email(test_email, mail_server.config.ip_address)
print(f"📧 Test phishing email: {'❌ Still delivered' if success else '✅ Blocked'}")

if not success:
    print("🛡️ Security measures are effective!")
    print("✅ Email blocking is working correctly")
else:
    print("⚠️ Security measures may need adjustment")

=== BLUE TEAM: Security Response ===
🔍 SOC analyst detects suspicious email activity...
   Suspicious sender: admin@company.com (from 192.168.1.100)
   Threat type: Email spoofing/phishing

🛡️ Implementing security countermeasures...
📤 Block sender request: success
✅ Sender blocked successfully
📊 Total blocked senders: 1
📤 Block IP request: success
✅ IP 192.168.1.100 blocked successfully
📊 Total blocked IPs: 1

🧪 Testing security measures...
📧 Test phishing email: ✅ Blocked
🛡️ Security measures are effective!
✅ Email blocking is working correctly


## Security Policy Variations

Demonstrate different security configurations and their effects.

In [7]:
print("=== SECURITY POLICY VARIATIONS ===")

# Show current security status
print("\n📊 Current Security Status:")
if hasattr(smtp_server, 'security_policy'):
    print(f"   🚫 Blocked senders: {len(smtp_server.security_policy.blocked_senders)}")
    print(f"   🚫 Blocked IPs: {len(smtp_server.security_policy.blocked_ips)}")
else:
    print("   ⚠️ No security policy configured")

# Demonstrate strict vs permissive mode
print("\n🔧 Testing different security configurations...")

# Count emails before and after
user_mailbox = smtp_server.mailbox_manager.get_mailbox("user")
finance_mailbox = smtp_server.mailbox_manager.get_mailbox("finance")
hr_mailbox = smtp_server.mailbox_manager.get_mailbox("hr")

user_count_before = len(user_mailbox.get_messages()) if user_mailbox else 0
finance_count_before = len(finance_mailbox.get_messages()) if finance_mailbox else 0
hr_count_before = len(hr_mailbox.get_messages()) if hr_mailbox else 0

print(f"\n📬 Email counts before additional tests:")
print(f"   User: {user_count_before} emails")
print(f"   Finance: {finance_count_before} emails")
print(f"   HR: {hr_count_before} emails")

# Try to send legitimate email from admin (should work)
legitimate_email = EmailMessage(
    sender="admin@company.com",
    recipients=["hr@company.com"],
    subject="Legitimate HR Notice",
    body="This is a legitimate email from the real admin."
)

success = admin_client.send_email(legitimate_email, mail_server.config.ip_address)
print(f"\n📧 Legitimate admin email: {'✅ Delivered' if success else '❌ Blocked'}")

# Show final email counts
user_count_after = len(user_mailbox.get_messages()) if user_mailbox else 0
finance_count_after = len(finance_mailbox.get_messages()) if finance_mailbox else 0
hr_count_after = len(hr_mailbox.get_messages()) if hr_mailbox else 0

print(f"\n📬 Final email counts:")
print(f"   User: {user_count_after} emails (+{user_count_after - user_count_before})")
print(f"   Finance: {finance_count_after} emails (+{finance_count_after - finance_count_before})")
print(f"   HR: {hr_count_after} emails (+{hr_count_after - hr_count_before})")

print("\n📊 Security Effectiveness Summary:")
blocked_attempts = 1 if not success else 0  # From the test email that should be blocked
delivered_attacks = user_count_after + finance_count_after - 2  # Subtract legitimate emails
print(f"   ✅ Blocked malicious attempts: {blocked_attempts}")
print(f"   ⚠️ Delivered attack emails: {max(0, delivered_attacks)}")
print(f"   🛡️ Security effectiveness: {'High' if blocked_attempts > delivered_attacks else 'Needs improvement'}")

=== SECURITY POLICY VARIATIONS ===

📊 Current Security Status:
   🚫 Blocked senders: 1
   🚫 Blocked IPs: 1

🔧 Testing different security configurations...

📬 Email counts before additional tests:
   User: 0 emails
   Finance: 0 emails
   HR: 0 emails

📧 Legitimate admin email: ❌ Blocked

📬 Final email counts:
   User: 0 emails (+0)
   Finance: 0 emails (+0)
   HR: 0 emails (+0)

📊 Security Effectiveness Summary:
   ✅ Blocked malicious attempts: 1
   ⚠️ Delivered attack emails: 0
   🛡️ Security effectiveness: High


## System Logs and Analysis

Review system activity using built-in logging methods.

In [8]:
print("=== SYSTEM LOGS AND ANALYSIS ===")

# Show mail server activity
print("\n📋 Mail Server Activity Log:")
mail_server.sys_log.show(last_n=10)

# Show attacker machine activity
print("\n📋 Attacker Machine Activity Log:")
attacker_pc.sys_log.show(last_n=5)

# Show user machine activity
print("\n📋 User Machine Activity Log:")
user_pc.sys_log.show(last_n=5)

# Final security summary
print("\n🔍 SECURITY ANALYSIS SUMMARY:")
print("   ✅ Multi-agent email scenarios completed")
print("   ✅ Attack simulation successful")
print("   ✅ Defense mechanisms tested")
print("   ✅ System logging captured all activities")
print("\n🎯 Key Learning Points:")
print("   • Email spoofing can be detected through source IP analysis")
print("   • Blue team responses can effectively block malicious senders")
print("   • Security policies need regular testing and adjustment")
print("   • System logs provide valuable forensic information")

=== SYSTEM LOGS AND ANALYSIS ===

📋 Mail Server Activity Log:
+---------------------------------------------------------------------------------------------+
|                                     mail_server Sys Log                                     |
+-------------------------+-------+-----------------------------------------------------------+
| Timestamp               | Level | Message                                                   |
+-------------------------+-------+-----------------------------------------------------------+
| 2025-09-26 20:47:40,795 | INFO  | smtp-server: Loaded 0/0 blocked IPs                       |
| 2025-09-26 20:47:40,795 | INFO  | smtp-server: Security logging: enabled                    |
| 2025-09-26 20:47:40,795 | INFO  | Starting service smtp-server                              |
| 2025-09-26 20:47:40,795 | INFO  | smtp-server: SMTP server started on port 25               |
| 2025-09-26 20:47:40,795 | INFO  | smtp-server: SMTP server started on po

In [9]:
# Returning to the original developer mode configuration.
PRIMAITE_CONFIG["developer_mode"]["enabled"] = original_dev_mode
PRIMAITE_CONFIG["developer_mode"]["output_sys_logs"] = original_sys_logs
PRIMAITE_CONFIG["developer_mode"]["output_agent_logs"] = original_agent_logs